In [1]:
import numpy as np
import pandas as pd
import dotenv
import os
import requests
from data_handler import DataHandler
dotenv.load_dotenv()

True

In [2]:
# get api key
key = os.getenv('ALPHA_KEY')
url = os.getenv('ALPHA_URL')

In [3]:
# test getting data for some tickers
# free tier only allows 1 ticker for an intraday request
params = {
    "function": "TIME_SERIES_INTRADAY",
    "interval": "1min",
    "apikey": key,
    "symbol": 'MSFT',
    "month": "2009-01",
    "outputsize": "compact",
    "extended_hours": True,
}
response = requests.get(url, params)
df = pd.DataFrame(response.json()['Time Series (1min)'])

params['month'] = '2009-02'
response = requests.get(url, params)
df = pd.concat([df, pd.DataFrame(response.json()['Time Series (1min)'])], axis=1).T

KeyError: 'Time Series (1min)'

In [ ]:
api = DataHandler("ALPHA")
df = api.get_data_raw(options={
    'symbol': 'VOO'
    'interval': '1min',
    'extended_hours': True,
    'adjusted': True
})

Getting data for: 2009-1
Got response: {'Information': 'We have detected your API key as JDH6YJS17ZHGNK31 and our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}


KeyError: 'Time Series (1min)'

In [50]:
# intraday unadjusted (adjusted is not free)
params = {
    "api_token": key,
    "symbols": ["AAPL"],
    "sort": "desc",
    "date_from": "2024-06-25",
    "date_to": "2024-06-30",
    "key_by_date": True,
}
response = requests.get(url_unadjusted, params)

In [51]:
# look at data
response_dict = dict(response.json())
response_dict.keys()
metadata = response_dict['meta']
data = response_dict['data']
metadata

{'date_from': '2025-07-25', 'date_to': '2025-08-01', 'max_period_days': 7}

In [52]:
# lets look at the data structure
df = pd.DataFrame(data)
print(df.columns)
print(df.head(5))

Index(['date', 'ticker', 'data'], dtype='object')
                       date ticker  \
0  2025-08-01T15:59:00.000Z   AAPL   
1  2025-08-01T15:58:00.000Z   AAPL   
2  2025-08-01T15:57:00.000Z   AAPL   
3  2025-08-01T15:56:00.000Z   AAPL   
4  2025-08-01T15:55:00.000Z   AAPL   

                                                data  
0  {'open': 202.38, 'high': 202.45, 'low': 202.19...  
1  {'open': 202.52, 'high': 202.54, 'low': 202.34...  
2  {'open': 202.62, 'high': 202.64, 'low': 202.37...  
3  {'open': 202.61, 'high': 202.71, 'low': 202.54...  
4  {'open': 202.77, 'high': 203.01, 'low': 202.44...  


In [53]:
# need to destructure the dataframe
df['data'][0].keys()
df_flat = pd.concat([df.drop(columns=["data"]), pd.json_normalize(df["data"])], axis=1)
df_flat.head(5)

,date,ticker,open,high,low,close,volume,is_extended_hours
0,2025-08-01T15:59:00.000Z,AAPL,202.38,202.45,202.19,202.37,33846,False
1,2025-08-01T15:58:00.000Z,AAPL,202.52,202.54,202.34,202.39,14766,False
2,2025-08-01T15:57:00.000Z,AAPL,202.62,202.64,202.37,202.54,13493,False
3,2025-08-01T15:56:00.000Z,AAPL,202.61,202.71,202.54,202.59,9979,False
4,2025-08-01T15:55:00.000Z,AAPL,202.77,203.01,202.44,202.59,16783,False


In [4]:
from data_handler import DataHandler
api = DataHandler()
df = api.get_data(["VOO"], date_from="2024-08-18", date_to="2025-08-18")

In [7]:
# look at data 
print(df.head()) 
print(len(df)) 
print("Earliest record: " + str(df['date'][0]))
print("Latest record: " + str(df['date'][1488]))
print("Number of records: " + str(len(df)))

   index                      date ticker    open    high     low   close  \
0      0  2025-07-28T09:30:00.000Z    VOO  586.01  586.19  585.91  586.19   
1      1  2025-07-28T09:31:00.000Z    VOO  586.12  586.12  586.08  586.08   
2      2  2025-07-28T09:32:00.000Z    VOO  585.92  586.04  585.92  586.04   
3      3  2025-07-28T09:33:00.000Z    VOO  585.99  586.14  585.96  586.08   
4      4  2025-07-28T09:34:00.000Z    VOO  586.12  586.12  585.96  586.04   

   volume  is_extended_hours  
0     501              False  
1     110              False  
2     121              False  
3    1629              False  
4    2352              False  
1628
Earliest record: 2025-07-28T09:30:00.000Z
Latest record: 2025-08-01T13:39:00.000Z
Number of records: 1628


In [12]:
# save to csv to save on api calls
api.save_to_csv("VOO_8_18_25_to_8_22_25", df)

True